In [ ]:
%matplotlib inline

from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.calculation.job.cp2k import Cp2kCalculation
from aiida.orm.data.structure import StructureData
from aiida.orm.data.parameter import ParameterData
from aiida.orm.code import Code

import ase.build
import matplotlib.pyplot as plt

In [ ]:
params = {
    "FORCE_EVAL": {
        'METHOD': 'Quickstep',
        'DFT': {
            'BASIS_SET_FILE_NAME': 'BASIS_MOLOPT',
            'QS': {
                'EPS_DEFAULT': 1.0e-12,
                'WF_INTERPOLATION': 'ps',
                'EXTRAPOLATION_ORDER': 3,
            },
            'MGRID': {
                'NGRIDS': 4,
                'CUTOFF': 280,
                'REL_CUTOFF': 30,
            },
            'XC': {
                'XC_FUNCTIONAL': {
                    '_': 'LDA',
                },
            },
            'POISSON': {
                'PERIODIC': 'none',
                'PSOLVER': 'MT',
            },
        },
        'SUBSYS': {
            'KIND': [
                {'_': 'O', 'BASIS_SET': 'DZVP-MOLOPT-SR-GTH', 'POTENTIAL': 'GTH-LDA-q6' },
                {'_': 'H', 'BASIS_SET': 'DZVP-MOLOPT-SR-GTH', 'POTENTIAL': 'GTH-LDA-q1' },
            ],
        },
    },
}

In [ ]:
inputs = {}

# code
inputs['code'] = Code.get_from_string("cp2k@daint")

# structure
atoms = ase.build.molecule('H2O')
atoms.center(vacuum=2.0)
inputs['structure'] = StructureData(ase=atoms)

# parameters
inputs['parameters'] = ParameterData(dict=params)

# options
inputs['_options'] = {
       "resources": {"num_machines": 1},
       "max_wallclock_seconds": 3 * 60, # 3 min
}
 
output = Cp2kCalculation.process().run(**inputs)

In [ ]:
output['output_parameters'].get_attrs()

In [ ]:
folder = output['retrieved'].folder.get_abs_path("path")
! ls $folder

In [ ]:
! cat $folder/aiida.out

In [ ]:
scf_lines = ! grep -e "Diag." $folder/aiida.out
scf_energies = [float(l.split()[5]) for l in scf_lines]
plt.plot(scf_energies)